In [ ]:
!pip install langchain pandas tiktoken openai pypdf faiss-cpu -q


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from pypdf import PdfReader

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
                                      chunk_size=1000,
                                      chunk_overlap=0)

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key="*******************************************")

reader = PdfReader("drive/MyDrive/Texts/rii-epub-biobanks-2015.pdf")
db = FAISS.from_texts([""], embeddings)
for page in reader.pages:
    text = page.extract_text()
    chunks = text_splitter.split_text(text)
    db.add_texts(chunks)
db.save_local("drive/MyDrive/FAISS", index_name="gpt_test")

In [ ]:
vector_store = FAISS.load_local("drive/MyDrive/FAISS", embeddings, "gpt_test")

In [ ]:
vector_store.similarity_search_with_score("роль биобанков")

[(Document(page_content='Еропкин М.Ю. Биобанки и их роль в системах биобезопасности, здравоохранения, биотехнологии, экологии и «экономике знаний»\nстратегии, нацеленные не только на финансирование биобанкинга, но и на извлечение прибыли из\nнего. \nОдин из руководителей создаваемого на базе коллекции Центра им. Алмазова биосервиса\nВиталий Пруцкий полагает, что успешность биобанка будет зависеть от оборота образцов: «Мы\nсчитаем,  что  это  должны  быть  гарантированно  востребованные  коллекции.  И,  как  во  всякой\nколлекции, здесь должны быть редкие экземпляры, которые понадобятся если не сейчас, то в\nбудущем, или образцы, сбор которых не требует больших затрат и которые можно себе позволить\nиметь. У нас нет цели заполнить тысячи квадратных метров рядами дьюаров, а есть цель наладить\nвысокий оборот данных. Говоря «биобанк» — мы упрощаем. В перспективе это будет именно\nбиосервис с функциями хранения и обработки. Мы рассчитываем на анализ данных методами', metadata={}),
  0.2300

In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0,
                 max_tokens=50, openai_api_key="*******************************************")
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)

In [ ]:
query = "какая роль биобанков"
chain(query)

{'query': 'какая роль биобанков',
 'result': 'Роль биобанков заключается в хранении и сохранении биологических образцов, таких как сыворотка крови, клетки и ткани, с'}

In [ ]:
query = "биобанки и безопасность"
chain(query)

{'query': 'биобанки и безопасность',
 'result': 'Биобанки играют важную роль в системе биологической безопасности. Они помогают предотвращать потенциальные и реальные б'}

In [ ]:
query = "день недели сегодня"
chain(query)

{'query': 'день недели сегодня',
 'result': 'Из предоставленного контекста невозможно определить текущий день недели.'}